# Extract stop locations from CTA data

 stops.txt - list of stop locations for bus and train, also includes parent station info for trains

 stop_id, latitude and longitude columns. Create a dictionary -> key: stop_id, value: (latitude, longitude)

 Location type. Valid options are:

0 (or blank) - Stop (or Platform). A location where passengers board or disembark from a transit vehicle. Is called a platform when defined within a parent_station.

1 - Station. A physical structure or area that contains one or more platform.

2 - Entrance/Exit. A location where passengers can enter or exit a station from the street. If an entrance/exit belongs to multiple stations, it may be linked by pathways to both, but the data provider must pick one of them as parent.

3 - Generic Node. A location within a station, not matching any other location_type, that may be used to link together pathways define in pathways.txt.

4 - Boarding Area. A specific location on a platform, where passengers can board and/or alight vehicles.


Latitude and longtitude of the location.

For stops/platforms (location_type=0) and boarding area (location_type=4), the coordinates must be the ones of the bus pole — if exists — and otherwise of where the travelers are boarding the vehicle (on the sidewalk or the platform, and not on the roadway or the track where the vehicle stops).

Conditionally Required:
- Required for locations which are stops (location_type=0), stations (location_type=1) or entrances/exits (location_type=2).
- Optional for locations which are generic nodes (location_type=3) or boarding areas (location_type=4).

In [ ]:
stops = pd.read_csv('/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/google_transit/stops.txt')
#stops.head()

In [ ]:
# Numeric color map
#color_map = {
#    0: "red",  # platform
#    1: "blue", # station
#    2: "green", # entrance/exit
#    3: "gray",  # generic node
#    4: "yellow",   # boarding area
#}

# Map 'location_type' to colors
#stops['color'] = stops['location_type'].map(color_map)

In [ ]:
#stops['color'] 

In [ ]:
"""Plot Stops"""

# Convert chicago GeoDataFrame's geometry to GeoJSON
geojson = chicago.geometry.__geo_interface__

# Create the choropleth map as the base
fig1 = px.choropleth_mapbox(chicago,
                           geojson=geojson,
                           locations=chicago.index,
                           color="constant",  # Specify a column for coloring, or use a constant value for a uniform color
                           mapbox_style="carto-positron",
                           height=800,
                           width=1200,
                           zoom=10,
                           center={"lat": stops['stop_lat'].mean(), "lon": stops['stop_lon'].mean()},
                           opacity=0.5,
                           hover_data={'pop': True})  # Adds population data to the hover tooltip)

# Add the CTA stops as a scatter layer on top of the choropleth map
"""fig1.add_scattermapbox(
    lat=stops['stop_lat'],
    lon=stops['stop_lon'],
    hoverinfo='text',
    hovertext=stops['stop_name'],
    marker=dict(size=5, color=stops['color'].tolist(), opacity=0.7),  
    mode='markers'
)

fig1.add_scattermapbox(
    lat=stops['stop_lat'],
    lon=stops['stop_lon'],
    hoverinfo='text',
    hovertext=stops['stop_name'],
    marker=dict(size=5, color=stops['color'].tolist(), opacity=0.7),  
    mode='markers'
)

fig1.add_scattermapbox(
    lat=[chicago.loc[14423]["INTPTLAT20"]],
    lon=[chicago.loc[14423]["INTPTLON20"]],
    hoverinfo='text',
    # Optionally, you can add specific hovertext for this block if you want
    hovertext=f'Block {14423}',
    marker=dict(size=20, color='brown', opacity=1),  # Making this block distinct with a different color and full opacity
    mode='markers',
    name='Highlighted Block'  # Optionally, give a name to this layer for clarity
)"""

# Show the updated map
#fig1.show()

In [ ]:
"""import plot as pl

height = 800

pl.add_routes(chicago, routes, height)"""

'import plot as pl\n\nheight = 800\n\npl.add_routes(chicago, routes, height)'

In [ ]:
#times_str_keys = {str(key): value for key, value in times.items()}

#with open("test_traveltime.json", "w") as outfile:
#    json.dump(times_str_keys, outfile)

In [ ]:
#import json
#import ast

#with open("test_traveltime.json", "r") as infile:
#    times_str_keys = json.load(infile)

# Convert string keys back to tuple keys with integers
#times = {ast.literal_eval(key): value for key, value in times_str_keys.items()}

# If you are sure keys are simple integer tuples, like "(1, 2)", you can also manually parse them:
# times = {tuple(map(int, key.strip("()").split(", "))) for key, value in times_str_keys.items()}


# CTA Data Information

agency.txt - basic agency information

routes.txt - route list with unique identifiers

trips.txt - has information about each trip taken by a CTA vehicle

stop_times.txt - scheduled arrival/departure times for each stop on each trip

calendar.txt - defines which service IDs operate on which days

calendar_dates.txt - has exceptions to calendar (such as a holiday when a Sunday schedule is operated)

shapes.txt - defines how to draw a route line on a map

frequencies.txt - defines headways when trips aren't specifically scheduled, but sent as-needed (such as service that follows a major event)

transfers.txt - has additional information to guide trip planners into identifying two stops as a transfer point and how much extra time to give people to transfer between services

# We need stop_times, routes, stops and trips. We first identify and store stops as attributes of nodes.

- The goal is to assign a travel time to each edge. We will use STOP_TIMES.TXT for that. Stop times are identified by trip id's. 

- STOP TIMES: We will parse ARRIVAL_TIME at STOPS for each TRIP_ID. Note that STOP_TIMES and TRIP_ID are directly related. 
    
- Then, we will calculate the travel time between consecutive stops by substracting the arrival times at the stops.

- Determine the nodes containing a stop in it (R-tree spatial indexing using STOP LOCATIONS). Assign an attribute called 'stop' = [stop_id] to each node. 
    
- If there is no stop in the Census block corresponding to a node, then 'stop' = [], empty list. For more than one stop, the list will have more than one stop_id.

# Stops are identified and saved. Now, we need to see the trips between identified stops.

- There might be two different trips between two nodes. In such case, each vehicle will have different travel time to travel through the edge connecting the nodes.

-  We assign multiple edges such that each edge represent a trip between two nodes. So, the construction of directed graph depends on trips.

-  Apply item 3 for a certain moment? We should take a normal distribution of daily travel times of each trip. 

- Should we consider waiting time? Dijkstra's algorithm may implicitly change many trips when determining the minimum shortest path connecting two nodes. 

- Which day should we consider? Monday? CALENDAR.TXT


In [ ]:
import pandas

calendar = pandas.read_csv('/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/google_transit/calendar.txt')

In [ ]:
calendar.head()

In [ ]:
frequencies = pandas.read_csv('/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/google_transit/frequencies.txt')
frequencies.head()

In [ ]:
routes = pandas.read_csv('/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/google_transit/routes.txt')
routes.head()

In [ ]:
# defines how to draw a route line on a map
shapes = pandas.read_csv('/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/google_transit/shapes.txt')
shapes.head()

In [ ]:
stop_times = pd.read_csv('/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/google_transit/stop_times.txt')
stop_times.head()

# trip_id: 	Identifies a trip.

# arrival_time: Arrival time at a specific stop for a specific trip on a route. If there are not separate times for arrival and departure at a stop,
#               enter the same value for arrival_time and departure_time. Scheduled stops where the vehicle strictly adheres to the specified arrival and
#               departure times are timepoints. If this stop is not a timepoint, it is recommended to provide an estimated or interpolated time. If this is not available, arrival_time can be left empty.

# stop_id: Identifies the serviced stop. All stops serviced during a trip must have a record in stop_times.txt. Referenced locations must be stops, not stations or station entrances.
#          A stop may be serviced multiple times in the same trip, and multiple trips and routes may service the same stop.

# stop_sequence: Order of stops for a particular trip. The values must increase along the trip but do not need to be consecutive.
#                Example: The first location on the trip could have a stop_sequence=1, the second location on the trip could have a stop_sequence=23, the third location could have a stop_sequence=40, and so on.

# pickup_type: Indicates pickup method. Valid options are:
#              0 or empty - Regularly scheduled pickup.
#              1 - No pickup available.
#              2 - Must phone agency to arrange pickup.
#              3 - Must coordinate with driver to arrange pickup.

# drop_off_type: 0 or empty - Regularly scheduled drop off.

# shape_dist_traveled: Actual distance traveled along the associated shape, from the first stop to the stop specified in this record. This field specifies how much of the shape to draw between any two stops during a trip.
#                      Must be in the same units used in shapes.txt. Values used for shape_dist_traveled must increase along with stop_sequence; they cannot be used to show reverse travel along a route.

In [ ]:
transfers = pd.read_csv('/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/google_transit/transfers.txt')
transfers.head()

# from_stop_id: Identifies a stop or station where a connection between routes begins. If this field refers to a station, the transfer rule applies to all its child stops.
# to_stop_id: Identifies a stop or station where a connection between routes ends. If this field refers to a station, the transfer rule applies to all child stops.
# transfer_type: Indicates the type of connection for the specified (from_stop_id, to_stop_id) pair.
#    0 or empty - Recommended transfer point between routes.
#    1 - Timed transfer point between two routes. The departing vehicle is expected to wait for the arriving one and leave sufficient time for a rider to transfer between routes.
#    2 - Transfer requires a minimum amount of time between arrival and departure to ensure a connection. The time required to transfer is specified by min_transfer_time.
#    3 - Transfers aren't possible between routes at the location.
#    4 - Passengers can stay onboard the same vehicle to transfer from one trip to another (an "in-seat transfer").
#    5 - In-seat transfers aren't allowed between sequential trips. The passenger must alight from the vehicle and re-board.

In [ ]:
transfers

In [ ]:
trips = pd.read_csv('/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/google_transit/trips.txt')
trips.head()

# route_id: Identifies a route.
# service_id: Identifies a set of dates when service is available for one or more routes.
# trip_id: Identifies a trip.
# direction_id: Indicates the direction of travel for a trip. 0 - Travel in one direction (e.g. outbound travel). 1 - Travel in the opposite direction (e.g. inbound travel).
# block_id: Identifies the block to which the trip belongs. A block consists of a single trip or many sequential trips made using the same vehicle,
#   defined by shared service days and block_id. A block_id can have trips with different service days, making distinct blocks.
# shape_id: Identifies a geospatial shape that describes the vehicle travel path for a trip.

In [ ]:
# Merge 'stop_times' with 'trips' to include 'route_id' and 'direction_id'
stop_times = stop_times.merge(trips[['trip_id', 'route_id', 'direction_id']], on='trip_id')

# Now, 'stop_times' DataFrame includes stop information, trip information, and direction
